In [6]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
data = pd.read_csv('adult.data.txt', header=None)
data.columns = np.array(['age', 'workclass', 'fnlwg', 'education', 'education.num', 'marital.status', 
                         'occupation','relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 
                         'hours.per.week', 'native.country', 'income'])

In [7]:
print(set(data['income']))
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1}).astype(int)

print(set(data['sex']))
data['sex'] = data['sex'].map({' Male': 0, ' Female': 1}).astype(int)

{' <=50K', ' >50K'}
{' Male', ' Female'}
